In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from IPython.display import display # used to display multiple command outputs in same cell

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/home-data-for-ml-course/sample_submission.csv
/kaggle/input/home-data-for-ml-course/sample_submission.csv.gz
/kaggle/input/home-data-for-ml-course/train.csv.gz
/kaggle/input/home-data-for-ml-course/data_description.txt
/kaggle/input/home-data-for-ml-course/test.csv.gz
/kaggle/input/home-data-for-ml-course/train.csv
/kaggle/input/home-data-for-ml-course/test.csv


##### *Loading test and train files*

In [3]:
# Editing the setting to show all the columns and rows
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [4]:
# Storing train and test csv file path
train_path = "../input/home-data-for-ml-course/train.csv"
test_path = "../input/home-data-for-ml-course/test.csv"

# loading train and test csv files into a panda dataframe
train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

### Analyzing train and test data

In [ ]:
# Finding number of rows and columns in train and test data
print("Training DataFrame")
display(train_df.shape)
print("Testing DataFrame")
display(test_df.shape)

In [ ]:
# Top 5 rows in train and test df
print("Training DataFrame Snippet")
display(train_df.head())
print("Testing DataFrame Snippet")
display(test_df.head())

In [ ]:
def create_summary(df):
#     Copying the dataframe passed into a different dataframe
    x = df.copy()
    
#     Creating empty dataframe
    summary_df = pd.DataFrame()
    
#     Storing Column names as values
    summary_df["columns"] = list(x.columns.values)
    
#     Adding data type of each column
    summary_df["data_type"] = ""
    
    for i in range(len(summary_df)):
        summary_df.loc[i, ["data_type"]] = x[summary_df.loc[i,"columns"]].dtypes
    
#     Adding number of rows for each columns
    summary_df["#_rows"] = len(x)
    
#     Number of null values in each columns
    summary_df["#_null_values"] = ""
    
    for i in range(len(summary_df)):
         summary_df.loc[i, ["#_null_values"]] = x[summary_df.loc[i,"columns"]].isnull().sum()
    
#     Number of unique values in each columns
    summary_df["#_unique_values"] = ""
    
    for i in range(len(summary_df)):
         summary_df.loc[i, ["#_unique_values"]] = len(x[summary_df.loc[i,"columns"]].unique())
    
#     Returning the summary dataframe
    return summary_df

In [ ]:
# Test and train summary
train_summary_df = create_summary(train_df)
test_summary_df = create_summary(test_df)

summary_df = pd.merge(train_summary_df, test_summary_df, how = "outer", on = "columns",
                    suffixes=['_train','_test'])

display(summary_df)

In [ ]:
# Description of all numerical columns in train and test data
print("Training DataFrame Numerical Column Description")
display(train_df.describe())
print("Testing DataFrame Numerical Column Description")
display(test_df.describe())

In [ ]:
# Code to specifically see what columns have null values and how many null values

# train_df.loc[:,train_df.columns[train_df.isnull().any()]].isnull().sum()
# test_df.loc[:,test_df.columns[test_df.isnull().any()]].isnull().sum()

In [31]:
x = train_df["MSSubClass"].unique().tolist()
x.sort()
print(x)
y = test_df["MSSubClass"].unique().tolist()
y.sort()
print(y)

[20, 30, 40, 45, 50, 60, 70, 75, 80, 85, 90, 120, 160, 180, 190]
[20, 30, 40, 45, 50, 60, 70, 75, 80, 85, 90, 120, 150, 160, 180, 190]


In [17]:
train_df["MSSubClass"].unique().tolist()

[60, 20, 70, 50, 190, 45, 90, 120, 30, 85, 80, 160, 75, 180, 40]

In [18]:
test_df["MSSubClass"].unique().tolist()

[20, 60, 120, 160, 80, 30, 50, 90, 85, 190, 45, 70, 75, 180, 40, 150]

In [32]:
import collections
compare = lambda x, y: collections.Counter(x) == collections.Counter(y)
compare(train_df["MSSubClass"].unique().tolist(), test_df["MSSubClass"].unique().tolist())

False

In [31]:
def buggy_column(x, y, a, b):
#     Storing arguements into a variable
    identifier = x
    predict = y
    train_df = a
    test_df = b
    
#     Finding column name in each data set
    train_df_columns = train_df.columns.values.tolist()
    test_df_columns = test_df.columns.values.tolist()
    
#     Checking if supplied identifier is in train and test dataframe
    if identifier not in train_df_columns:
        return print("Given identifer column is not present in train dataframe")
    if identifier not in test_df_columns:
        return print("Given identifer column is not present in test dataframe")

#     Checking if predict column is present in the train_df
    if predict not in train_df_columns:
        return print("Given column to predict is not present in train dataframe")
    
#     Checking if all the columns of train_df is present in test_df
    not_present_columns = []
    for i in train_df_columns:
        if (i not in test_df_columns) or (i != predict):
            not_present_columns.append(i)
        print("Above train dataframe columns are missing from test dataframe")
        print(not_present_columns)
    
#     Checing if all the values in each column of train and test are present in each other
    buggy_columns = []
    for i in train_df_columns:
        if i not in not_present_columns:
            
        

    
    return

In [33]:
# Identify the columns in test and train data for whom values dont match

buggy_column("columns", "a", train_df, test_df)

Given identifer column is not present in train dataframe
Given identifer column is not present in train dataframe


### Value imputation

In [ ]:
# MiscFeature can be dropped as we dont know the nature of data
# Creating a copy of tran and test data and dropping MiscFeature column in both